Notebook to study PCA on our animations

In [65]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%pylab inline

from matplotlib import pyplot as plt
from matplotlib import animation
#plt.rcParams['animation.ffmpeg_path'] = 'C:\\ffmpeg\\bin\\'

VIDEO_TAG = """<video controls>
 <source src="data:video/x-m4v;base64,{0}" type="video/mp4">
 Your browser does not support the video tag.
</video>"""

def anim_to_html(anim, fps=120):
    if not hasattr(anim, '_encoded_video'):
        anim.save("test.mp4", fps=fps, extra_args=['-vcodec', 'libx264', '-pix_fmt', 'yuv420p'])

        video = open("test.mp4","rb").read()

    anim._encoded_video = video.encode("base64")
    return VIDEO_TAG.format(anim._encoded_video)

from IPython.display import HTML

def display_animation(anim, fps=120):
    plt.close(anim._fig)
    return HTML(anim_to_html(anim, fps=fps))

hi


In [81]:
import vicon_anim_parser.scene_drawer as sd
import Learning.data_cleaning as dc
import numpy as np

In [ ]:
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 8, 6

In [6]:
segment_id2parent_id, header, anim_data = dc.get_segment_anim_data("Ruslan.vsk", "./SwordMovements/sword_movements_01.csv")

In [9]:
def gather_data_suitable_for_PCA(anim_data):
    seq1 = dc.convert_segment_anim_for_learning(anim_data[540:3190])
    seq2 = dc.convert_segment_anim_for_learning(anim_data[4170:4800])
    seq3 = dc.convert_segment_anim_for_learning(anim_data[5900:7200])
    
    print seq1.shape
    print seq2.shape
    print seq3.shape
    
    return np.concatenate((seq1, seq2, seq3), axis=0)

input_PCA = gather_data_suitable_for_PCA(anim_data)
print input_PCA.shape

(2650L, 87L)
(630L, 87L)
(1300L, 87L)
(4580L, 87L)


In [46]:
# SVD decompositon exmaple (PCA)
X = np.arange(16).reshape(4, 4)
print X
mean_X = np.mean(X, axis=0)
print mean_X
print mean_X.shape
mean_X_box = np.dot(np.ones((4, 1)), mean_X.reshape(1, 4))
print mean_X_box
centered_X = X - mean_X_box
print centered_X
scatter = np.dot(centered_X.T, centered_X)
print scatter
U, D, Vt = np.linalg.svd(scatter)
print "D:"
print D
print
print Vt[0, :]
print Vt[1, :]
print Vt[2, :]
print Vt[3, :]
print 
PCA_number = 1
decoded = np.dot(np.dot(centered_X, Vt.T[:, 0:PCA_number]), Vt[0:PCA_number, :]) + mean_X_box
print decoded
print Vt.T[:, 0:4]

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]
[ 6.  7.  8.  9.]
(4L,)
[[ 6.  7.  8.  9.]
 [ 6.  7.  8.  9.]
 [ 6.  7.  8.  9.]
 [ 6.  7.  8.  9.]]
[[-6. -6. -6. -6.]
 [-2. -2. -2. -2.]
 [ 2.  2.  2.  2.]
 [ 6.  6.  6.  6.]]
[[ 80.  80.  80.  80.]
 [ 80.  80.  80.  80.]
 [ 80.  80.  80.  80.]
 [ 80.  80.  80.  80.]]
D:
[  3.20000000e+02   6.66133815e-15   5.72643226e-46   1.84848770e-78]

[-0.5 -0.5 -0.5 -0.5]
[ 0.8660254  -0.28867513 -0.28867513 -0.28867513]
[ 0.          0.81649658 -0.40824829 -0.40824829]
[  0.00000000e+00  -4.42903565e-17  -7.07106781e-01   7.07106781e-01]

[[ -3.55271368e-15   1.00000000e+00   2.00000000e+00   3.00000000e+00]
 [  4.00000000e+00   5.00000000e+00   6.00000000e+00   7.00000000e+00]
 [  8.00000000e+00   9.00000000e+00   1.00000000e+01   1.10000000e+01]
 [  1.20000000e+01   1.30000000e+01   1.40000000e+01   1.50000000e+01]]
[[ -5.00000000e-01   8.66025404e-01   0.00000000e+00   0.00000000e+00]
 [ -5.00000000e-01  -2.88675135e-01   8.16496

In [122]:
def convert_to_box(vector, n, d):
    return np.dot(np.ones((n, 1)), vector.reshape(1, d))

def center_data(X, mean_X = None):
    n, d = X.shape
    if mean_X is None:
        mean_X = np.mean(X, axis=0)
    mean_X_box = convert_to_box(mean_X, n, d)
    centered_X = X - mean_X_box   
    return centered_X

def perform_decomposition(X):
    #data matrix of dimensions n x d
    #n - number of learning instances
    #d - dimensionality of data point
    n, d = X.shape
    mean_X = np.mean(X, axis=0)
    centered_X = center_data(X, mean_X)
    scatter = np.dot(centered_X.T, centered_X)
    U, D, Vt = np.linalg.svd(scatter)
    return mean_X, D, Vt.T

def show_D_plot(D):
    sum_D = sum(D)
    xx = range(1, len(D) + 1)
    fx = [sum(D[:x])/sum_D for x in xx]
    dc.plot_styled(xx, fx, label_x="number of PCA", label_fx="variance covered")
    
def calc_PCA_number(D, variance_preserved=0.97):
    sum_D = sum(D)
    xx = range(1, len(D) + 1)
    for PCA_number in xx:
        variance_covered = sum(D[:PCA_number])/sum_D
        if variance_covered > variance_preserved:
            return PCA_number
    return len(D)

def project_data(X, V, mean_X=None):
    """
    project data in the matrix form X onto
    V - the column based matrix of PCA
    """
    centered_X = center_data(X, mean_X)
    return np.dot(centered_X, V)

def unproject_data(projected_X, V, mean_X):
    """
    V - is the column based matrix of PCA
    returns data in the matrix form
    """
    n = projected_X.shape[0]
    d = mean_X.shape[0]
    mean_X_box = convert_to_box(mean_X, n, d)
    return np.dot(projected_X, V.T) + mean_X_box

def unproject_single(projected_pose, V, mean_X):
    assert len(projected_pose.shape) == 1, "input projected_pose has more then 1 dimension %s" % projected_pose.shape
    d = projected_pose.shape[0]
    unproject_pose = unproject_data(projected_pose.reshape(1, d), V, mean_X)
    #convert into vector
    return unproject_pose.reshape(unproject_pose.shape[1])

In [93]:
mean_pose, pose_D, pose_V = perform_decomposition(input_PCA)

In [153]:
show_D_plot(pose_D)

In [94]:
print len(mean_pose)

87


In [96]:
segments_mean_pose = dc.convert_pose_vector_to_segments(mean_pose)
sd.SegmentsDrawer.show_pose(segments_mean_pose, segment_id2parent_id, scene_length=800)

In [97]:
num_PCA_95 = calc_PCA_number(pose_D, 0.95)
print "horizontal animation required number of PCA to cover 95%:", num_PCA_95

horizontal animation required number of PCA to cover 95%: 3


In [106]:
PCA_95 = pose_V[:, :num_PCA_95]

In [107]:
projected_anim_data = project_data(input_PCA, PCA_95, mean_pose)
print projected_anim_data.shape

(4580L, 3L)


In [135]:
projected_std = np.std(projected_anim_data, axis=0)
projected_mean = np.mean(projected_anim_data, axis=0)
print projected_std
print projected_mean

[ 958.97583646  511.06156705  242.13679637]
[ -4.54191520e-04   3.63112021e-05   8.80066606e-04]


In [152]:
pose_0 = unproject_single(np.array([958.97583646, -582.14850442, 0]), PCA, mean_pose)
sd.SegmentsDrawer.show_pose(dc.convert_pose_vector_to_segments(pose_0), segment_id2parent_id, scene_length=800)

In [136]:
print np.max(projected_anim_data[:, 0]), np.argmax(projected_anim_data[:, 0])

1582.84292411 2378


In [138]:
print projected_anim_data[2378]

[ 1582.84292411  -582.14850442   325.54551949]


In [147]:
pose_1 = unproject_single(projected_anim_data[2378], PCA, mean_pose)
sd.SegmentsDrawer.show_pose(dc.convert_pose_vector_to_segments(pose_1), segment_id2parent_id, scene_length=800)